In [ ]:
import os
import json
import numpy as np
import matplotlib.pyplot as plt
import json

from disentangle.data_loader.pavia3_rawdata_loader import load_one_file
from disentangle.core.tiff_reader import save_tiff




In [ ]:
root_input_dir = '/group/jug/ashesh/data/pavia4_HighSNR'
root_output_dir = '/group/jug/ashesh/data/pavia4_HighSNR_cropped'
condition = 'Deconvolved/Cond_3/Divided_4/'
crop_and_save = True
manually_find_bbox = not crop_and_save

input_dir = os.path.join(root_input_dir, condition)
fnames = list(os.listdir(input_dir))
print('Number of files', len(fnames))
output_dir = os.path.join(root_output_dir, condition)
os.makedirs(output_dir, exist_ok=True)
output_bbox_fpath = os.path.join(output_dir, 'crop_dict.json')

assert not (crop_and_save and manually_find_bbox), 'Cannot crop and manually find bbox at the same time'
if manually_find_bbox:
    crop_dict = {}
elif  crop_and_save:
    print('Loading the bbox dict', output_bbox_fpath)
    with open(output_bbox_fpath, 'r') as f:
        crop_dict = json.load(f)
    assert len(crop_dict) == len(fnames), 'Number of files in crop dict and input dir do not match'


In [ ]:
idx = 4
fname = fnames[idx]
input_fpath = os.path.join(input_dir, fname)
output_fpath = os.path.join(output_dir, fname.replace('.nd2', '.tif'))
data = load_one_file(input_fpath)
print(fname, data.shape)

In [ ]:
if manually_find_bbox:
    hs = 500#0
    he = 2720

    ws = 0
    we = 2720
    cropped_data = data[:,hs:he, ws:we]
    crop_dict[fname] = (hs, he, ws, we)
    _,ax = plt.subplots(figsize=(12,6),ncols=4,nrows=2)
    ax[0,0].imshow(cropped_data[0,...,0])
    ax[0,1].imshow(cropped_data[2,...,0])
    ax[0,2].imshow(cropped_data[4,...,0])
    ax[0,3].imshow(cropped_data[6,...,0])

    ax[1,0].imshow(cropped_data[0,...,1])
    ax[1,1].imshow(cropped_data[2,...,1])
    ax[1,2].imshow(cropped_data[4,...,1])
    ax[1,3].imshow(cropped_data[6,...,1])

elif crop_and_save:
    hs, he, ws, we = crop_dict[fname]
    cropped_data = data[:,hs:he, ws:we]
    plt.imshow(cropped_data[1,...,0])
    save_tiff(output_fpath, cropped_data)
    print('Saved', output_fpath)

In [ ]:
if manually_find_bbox:
    print('Writing to', output_bbox_fpath)
    print(crop_dict)
    assert len(crop_dict) == len(fnames)
    with open(output_bbox_fpath, 'w') as f:
        json.dump(crop_dict, f)